<a href="https://colab.research.google.com/github/chaitravi-ce/PyTorch_Tutorials/blob/main/PyTorch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch

# **Training From Scratch**

## Training Data

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [ ]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [ ]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs.shape)
print(targets.shape)

torch.Size([5, 3])
torch.Size([5, 2])


In [ ]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.7207, -2.2588, -0.0666],
        [ 1.2553, -0.7123, -0.4710]], requires_grad=True)
tensor([ 0.5945, -0.3529], requires_grad=True)


In [ ]:
# @ represents matrix multiplication and .t() transpose

def model(x):
    return x @ w.t() + b

In [ ]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-206.2175,   23.3082],
        [-268.0228,   21.0542],
        [-368.6463,  -13.9053],
        [-172.5054,   79.6333],
        [-270.6384,  -15.0873]], grad_fn=<AddBackward0>)


## Loss Function

In [ ]:
# MSE loss

# .numel() to get no. of elements
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [ ]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(65576.1562, grad_fn=<DivBackward0>)


## Compute Gradients

In [ ]:
loss.backward()

In [ ]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.7207, -2.2588, -0.0666],
        [ 1.2553, -0.7123, -0.4710]], requires_grad=True)
tensor([[-27789.7598, -31572.0410, -19049.5391],
        [ -5673.5513,  -8177.6069,  -4690.6924]])


In [ ]:
print(b)
print(b.grad)

tensor([ 0.5945, -0.3529], requires_grad=True)
tensor([-333.4061,  -72.9994])


In [ ]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Training the Model

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

In [ ]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-206.2175,   23.3082],
        [-268.0228,   21.0542],
        [-368.6463,  -13.9053],
        [-172.5054,   79.6333],
        [-270.6384,  -15.0873]], grad_fn=<AddBackward0>)


In [ ]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(65576.1562, grad_fn=<DivBackward0>)


In [ ]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-27789.7598, -31572.0410, -19049.5391],
        [ -5673.5513,  -8177.6069,  -4690.6924]])
tensor([-333.4061,  -72.9994])


In [ ]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [ ]:
print(w)
print(b)

tensor([[-0.4428, -1.9431,  0.1239],
        [ 1.3121, -0.6305, -0.4241]], requires_grad=True)
tensor([ 0.5978, -0.3522], requires_grad=True)


In [ ]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(45059.4922, grad_fn=<DivBackward0>)


## Train for multiple epochs

In [ ]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [ ]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(254.5292, grad_fn=<DivBackward0>)


In [ ]:
print(preds, targets)

tensor([[ 60.9939,  75.9291],
        [ 88.1113,  98.5773],
        [ 99.1112, 128.6822],
        [ 43.0803,  69.4329],
        [ 99.4739,  96.5089]], grad_fn=<AddBackward0>) tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


# Linear regression using PyTorch built-ins

In [ ]:
import torch.nn as nn

In [ ]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

## Dataset and DataLoader

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds

In [ ]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [ ]:
for x, y in train_dl:
    print(x)
    print(y)
    break

tensor([[ 91.,  87.,  65.],
        [ 87., 135.,  57.],
        [ 68.,  96.,  71.],
        [101.,  44.,  37.],
        [ 73.,  67.,  43.]])
tensor([[ 80., 102.],
        [118., 134.],
        [104., 118.],
        [ 21.,  38.],
        [ 56.,  70.]])


In [ ]:
train_ds[[1,2,3]]

(tensor([[ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.]]), tensor([[ 81., 101.],
         [119., 133.],
         [ 22.,  37.]]))

## nn.Linear Module

In [ ]:
# Define model

# 3 inputs and 2 outputs
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2697,  0.5268,  0.1700],
        [-0.5241, -0.4857,  0.5195]], requires_grad=True)
Parameter containing:
tensor([ 0.3490, -0.4168], requires_grad=True)


In [ ]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.2697,  0.5268,  0.1700],
         [-0.5241, -0.4857,  0.5195]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3490, -0.4168], requires_grad=True)]

In [ ]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 23.2636, -48.8858],
        [ 33.0406, -57.6120],
        [ 57.3306, -80.9762],
        [  1.7804, -55.5445],
        [ 44.2078, -46.8504],
        [ 22.4671, -48.9242],
        [ 32.6838, -56.6068],
        [ 57.2309, -80.9809],
        [  2.5768, -55.5061],
        [ 44.6474, -45.8068],
        [ 22.9068, -47.8806],
        [ 32.2442, -57.6504],
        [ 57.6874, -81.9814],
        [  1.3407, -56.5880],
        [ 45.0042, -46.8120]], grad_fn=<AddmmBackward>)

## Loss Functions

In [ ]:
import torch.nn.functional as F

In [ ]:
loss_fn = F.mse_loss

In [ ]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(13199.4033, grad_fn=<MseLossBackward>)


In [ ]:
?nn.Linear

## Optimizer

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [ ]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [ ]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 303.2576
Epoch [20/100], Loss: 154.5360
Epoch [30/100], Loss: 86.4176
Epoch [40/100], Loss: 46.0727
Epoch [50/100], Loss: 36.1146
Epoch [60/100], Loss: 10.6485
Epoch [70/100], Loss: 30.9729
Epoch [80/100], Loss: 12.0309
Epoch [90/100], Loss: 13.8786
Epoch [100/100], Loss: 11.5820


In [ ]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.7637,  70.9912],
        [ 80.4817, 102.0675],
        [121.2221, 128.3032],
        [ 24.3498,  41.7759],
        [ 97.1122, 118.7369],
        [ 56.5426,  70.0738],
        [ 79.9959, 102.3894],
        [121.3452, 129.0687],
        [ 25.5710,  42.6933],
        [ 97.8476, 119.9761],
        [ 57.2779,  71.3131],
        [ 79.2605, 101.1502],
        [121.7079, 127.9813],
        [ 23.6145,  40.5367],
        [ 98.3334, 119.6542]], grad_fn=<AddmmBackward>)

In [ ]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.9205, 68.7979]], grad_fn=<AddmmBackward>)